In [1]:
import geff

In [2]:
meta = geff.GeffMetadata(
    axes=[
        {"name": "frame", "type": "time", "unit": "frame"},
        {"name": "y", "type": "space", "unit": "px"},
        {"name": "x", "type": "space", "unit": "px"},
    ],
    related_objects=[
        {"type": "image", "path": "../image"},
        {"type": "labels", "path": "../labels", "label_prop": "seg_id"},
    ],
    track_node_props={"lineage": "tree_id", "tracklet": "track_id"},
    directed=True,
)
with open("metadata.json", "w") as f:
    f.write(meta.model_dump_json(indent=2))

/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/_internal/_validators.py:52: UserWarning: Temporal unit frame not in valid OME-Zarr units [None, 'attosecond', 'centisecond', 'day', 'decisecond', 'exasecond', 'femtosecond', 'gigasecond', 'hectosecond', 'hour', 'kilosecond', 'megasecond', 'microsecond', 'millisecond', 'minute', 'nanosecond', 'petasecond', 'picosecond', 'second', 'terasecond', 'yoctosecond', 'yottasecond', 'zeptosecond', 'zettasecond']. Reader applications may not know what to do with this information.
  v_list = validator(input_value)
/Users/fukai/projects/laptrack/.venv/lib/python3.12/site-packages/pydantic/_internal/_validators.py:52: UserWarning: Spatial unit px not in valid OME-Zarr units [None, 'angstrom', 'attometer', 'centimeter', 'decimeter', 'exameter', 'femtometer', 'foot', 'gigameter', 'hectometer', 'inch', 'kilometer', 'megameter', 'meter', 'micrometer', 'mile', 'millimeter', 'nanometer', 'parsec', 'petameter', 'picometer', 'teramet

In [3]:
from skimage.measure import regionprops_table

import pandas as pd
from laptrack import datasets
import zarr

In [9]:
images, labels = datasets.cell_segmentation()
regionprops = []
for frame, label in enumerate(labels):
    df = pd.DataFrame(regionprops_table(label, properties=["label", "centroid"]))
    df["frame"] = frame
    regionprops.append(df)
regionprops_df = pd.concat(regionprops).rename(
    columns={"centroid-0": "y", "centroid-1": "x"}
)

In [10]:
ds = zarr.open("images.zarr", mode="w")
ds.create_array("image", data=images)
ds.create_array("labels", data=labels)

<Array file://images.zarr/labels shape=(10, 234, 234) dtype=uint8>

In [11]:
ds["image"].shape

(10, 234, 234)

In [12]:
regionprops_df.to_csv("regionprops.csv", index=False)